# GraphSAGE - Link Stealing Attack
> Steal links from GraphSAGE model

## Imports

In [1]:
import argparse
import time
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl import DGLGraph
from dgl.data import register_data_args
from dgl.data import citation_graph as citegrh
from dgl.nn.pytorch.conv import SAGEConv
import matplotlib.pyplot as plt

Using backend: pytorch


#### Random

In [2]:
import random
random.seed(1)

#### Disable Warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Target Model

Inductive trained label prediction model 

#### GraphSAGE model

In [4]:
class GraphSAGE(nn.Module):
    
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self.activation = activation

        # input layer
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type))
        # output layer
        self.layers.append(SAGEConv(n_hidden, n_classes, aggregator_type)) # activation None

        
    def forward(self, graph, inputs):
        h = self.dropout(inputs)
        for l, layer in enumerate(self.layers):
            h = layer(graph, h)
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h

#### Classification

In [5]:
def evaluate(model, graph, features, labels, nid):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[nid]
        labels = labels[nid]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
def load_data(args):
    if args['dataset'] == 'cora':
        return citegrh.load_cora()
    elif args['dataset'] == 'citeseer':
        return citegrh.load_citeseer()
    elif args['dataset'] == 'pubmed':
        return citegrh.load_pubmed()
    elif args['dataset'] is not None and args['dataset'].startswith('reddit'):
        return RedditDataset(self_loop=('self-loop' in args.dataset))
    else:
        raise ValueError('Unknown dataset: {}'.format(args.dataset))

In [11]:
def main(args, process_output=False):
    # load and preprocess dataset
    print('---------Load Dataset----------\n')

    data = load_data(args)
    g = data[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    in_feats = features.shape[1]
    n_classes = data.num_classes
    n_edges = data.graph.number_of_edges()
    
    print("""\n\n--------Data statistics--------
    
  Edges   %d
  Classes %d
    
  Train samples %d
  Val samples   %d
  Test samples  %d""" %
          (n_edges, n_classes,
           train_mask.int().sum().item(),
           val_mask.int().sum().item(),
           test_mask.int().sum().item()))

    if args['gpu'] < 0:
        cuda = False
    else:
        cuda = True
        torch.cuda.set_device(args['gpu'])
        features = features.cuda()
        labels = labels.cuda()
        train_mask = train_mask.cuda()
        val_mask = val_mask.cuda()
        test_mask = test_mask.cuda()
        print("\n  Cuda in use", args['gpu'])

    train_nid = train_mask.nonzero().squeeze()
    val_nid = val_mask.nonzero().squeeze()
    test_nid = test_mask.nonzero().squeeze()

    # graph preprocess and calculate normalization factor
    g = dgl.remove_self_loop(g)
    n_edges = g.number_of_edges()
    if cuda:
        g = g.int().to(args['gpu'])

    # create GraphSAGE model
    model = GraphSAGE(in_feats,
                      args['n_hidden'],
                      n_classes,
                      args['n_layers'],
                      F.relu,
                      args['dropout'],
                      args['aggregator_type'])

    if cuda:
        model.cuda()

    # use optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

    # initialize graph
    dur = []
    if process_output:
        print("\n\n--------Training process--------\n")
    for epoch in range(args['n_epochs']):
        model.train()
        if epoch >= 3:
            t0 = time.time()
        # forward
        logits = model(g, features)
        loss = F.cross_entropy(logits[train_nid], labels[train_nid])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch >= 3:
            dur.append(time.time() - t0)

        acc = evaluate(model, g, features, labels, val_nid)
        if process_output:
            print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
              "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur), loss.item(),
                                            acc, n_edges / np.mean(dur) / 1000))

    print("\n\n----------Test Result----------\n")
    acc = evaluate(model, g, features, labels, test_nid)
    print("Test Accuracy {:.4f}".format(acc))
    
    return model, data

In [12]:
# Model Parameter
para = {'aggregator_type': 'gcn', 
        'dataset': 'cora', 
        'dropout': 0.5, 
        'gpu': 0, 
        'lr': 0.01, 
        'n_epochs': 200, 
        'n_hidden': 16, 
        'n_layers': 2, 
        'weight_decay': 0.0005}

target_model, data = main(para)

---------Load Dataset----------

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


--------Data statistics--------
    
  Edges   10556
  Classes 7
    
  Train samples 140
  Val samples   500
  Test samples  1000

  Cuda in use 0


----------Test Result----------

Test Accuracy 0.8200


## Attacker Model

Fully connected Neural Network

In [9]:
# Define model
class FNN(nn.Module):
    
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super(FNN, self).__init__()
        self.layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self.activation = activation

        # input layer
        self.layers.append(nn.Linear(in_feats, n_hidden))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(Linear(n_hidden, n_hidden))
        # output layer
        self.layers.append(Linear(n_hidden, n_classes)) # activation None
        
    def forward(self, graph, inputs):
        h = self.dropout(inputs)
        for l, layer in enumerate(self.layers):
            h = layer(graph, h)
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h

## Link Stealing

In [25]:
# Target Model Posterior of one node
cora_graph = data[0]
features = cora_graph.ndata['feat']
labels = cora_graph.ndata['label']
val_mask = cora_graph.ndata['val_mask']
val_nid = val_mask.nonzero().squeeze()

torch.cuda.set_device(0)
cora_graph = cora_graph.int().to(0)
features = features.cuda()
labels = labels.cuda()
val_mask = val_mask.cuda()

In [47]:
def get_posterior(model, graph, features, labels, ind):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[ind]
        labels = labels[ind]
        print(logits, labels)

In [50]:
get_posterior(target_model, cora_graph, features, labels, 0)

tensor([-0.4448, -0.8945,  0.9963,  7.3860,  0.0137, -8.8878,  0.1598],
       device='cuda:0') tensor(3, device='cuda:0')
